## FINAL PROJECT: ISM 6562 - BIG DATA FOR BUSINESS

### **Setup Steps**

> ⚠️ **Note:** You must have a Java Development Kit (JDK) installed and the `JAVA_HOME` environment variable correctly configured.

- **For Windows**: The stable JDK version as of May 3, 2025, is **Java 11**.  
- **For macOS**: The stable version is **Java 17**, available from [Adoptium](https://adoptium.net/).

The Python code below checks for required packages and installs any that are missing. This ensures all necessary libraries are available for import in your project.


In [37]:
import importlib.util
import subprocess
import sys

# List of packages to ensure are installed
packages = [
    "numpy",
    "pandas",
    "matplotlib",
    "seaborn",
    "ucimlrepo",
    "pyspark",
    "sklearn",       # scikit-learn's import name is 'sklearn'
    "findspark"
]

for package in packages:
    if importlib.util.find_spec(package) is None:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    else:
        print(f"{package} is already installed.")


numpy is already installed.
pandas is already installed.
matplotlib is already installed.
seaborn is already installed.
ucimlrepo is already installed.
pyspark is already installed.
sklearn is already installed.
findspark is already installed.


In [38]:
# Importing libraries
import findspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo 
from pyspark.sql import SparkSession

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

sns.color_palette(palette='viridis')

[(0.275191, 0.194905, 0.496005),
 (0.212395, 0.359683, 0.55171),
 (0.153364, 0.497, 0.557724),
 (0.122312, 0.633153, 0.530398),
 (0.288921, 0.758394, 0.428426),
 (0.626579, 0.854645, 0.223353)]

### Import Data from UCI ML dataset library
Importing as a pandas dataframe as this is how the data is accessible from the UCI ML Repo. For cases where there was a larger dataset, a new pipeline into pyspark would further need to be developed. 

In [39]:
findspark.init() 
# If you have a SPARK_HOME environment variable set, it might find it automatically
spark_path = findspark.find()
print(spark_path)

c:\Users\baker\anaconda3\Lib\site-packages\pyspark


In [40]:
# fetch dataset 
communities_and_crime = fetch_ucirepo(id=183) 

# data (as pandas dataframes) 
X = communities_and_crime.data.features 
y = communities_and_crime.data.targets 

df = pd.concat([X, y], axis=1)

##### Create Spark Session

In [41]:
spark_df.show()

+-----+------+---------+--------------------+----+----------+-------------+------------+------------+------------+-----------+-----------+-----------+-----------+----------+---------+--------+---------+--------+------------+----------+----------+-----------+----------+---------+---------+-----------+-----------+------------+-----------+-----------+----------+-----------+--------------+---------------+------------+-----------+-------------+---------+-----------+---------------+------------+----------------+--------------+--------------+------------+-----------+----------+----------+-----------+----------------+-----------+-------------------+----------+--------+--------+--------+--------------+------------+------------+-------------+--------------+------------+------------+-------------+----------------+-------------------+---------------+-----------------+----------------+-----------------+------------------+---------------+----------------+--------------+--------+----------+----------

In [42]:
spark_df.createOrReplaceTempView("raw_crime")

In [43]:
columns = spark_df.columns

sql_expr = ",\n".join(
    [f"NULLIF({col}, '?') AS {col}" for col in columns]
)

query = f"SELECT {sql_expr} FROM raw_crime"
clean_df = spark.sql(query)
clean_df.createOrReplaceTempView("clean_crime")

#### Cleaning Dataset
Removing columns that are identifiers but will not provide any meaningful predictive values. Fold column was originally used as the cross-validation in the dataset.

In [44]:
cols_to_drop = ["state", "county", "community", "communityname", "fold"]
selected_cols = [c for c in clean_df.columns if c not in cols_to_drop]
clean_df = clean_df.select(*selected_cols)
clean_df.createOrReplaceTempView("final_crime")

In [45]:
numeric_cast_expr = ",\n".join(
    [f"CAST({col} AS DOUBLE) AS {col}" for col in selected_cols]
)

casted_df = spark.sql(f"SELECT {numeric_cast_expr} FROM final_crime")
casted_df.createOrReplaceTempView("tidy_crime")

In [46]:
# This is your clean, tidy, and numeric Spark DataFrame
tidy_df = spark.sql("SELECT * FROM tidy_crime")
tidy_df.show()

+----------+-------------+------------+------------+------------+-----------+-----------+-----------+-----------+----------+---------+--------+---------+--------+------------+----------+----------+-----------+----------+---------+---------+-----------+-----------+------------+-----------+-----------+----------+-----------+--------------+---------------+------------+-----------+-------------+---------+-----------+---------------+------------+----------------+--------------+--------------+------------+-----------+----------+----------+-----------+----------------+-----------+-------------------+----------+--------+--------+--------+--------------+------------+------------+-------------+--------------+------------+------------+-------------+----------------+-------------------+---------------+-----------------+----------------+-----------------+------------------+---------------+----------------+--------------+--------+----------+------------+-------------+----------------+--------------+

##### VectorAssembler
Use case: Combine multiple numeric features into a single feature vector column.

Why it's useful: This is a prerequisite for feeding data into ML models like linear regression or random forest.

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[col for col in df.columns if col != 'target'], 
    outputCol='features'
)
assembled_df = assembler.transform(df)


##### StandardScaler
Use case: Normalize features to have zero mean and unit variance.

Why it's useful: Helps gradient-based models like logistic regression or linear regression converge better.

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=True)
scaled_df = scaler.fit(assembled_df).transform(assembled_df)


##### PCA (Principal Component Analysis)
Use case: Dimensionality reduction.

Why it's useful: The dataset has 100+ features, many of which are correlated.

In [ ]:
from pyspark.ml.feature import PCA

pca = PCA(k=20, inputCol='scaledFeatures', outputCol='pcaFeatures')
pca_model = pca.fit(scaled_df)
result_df = pca_model.transform(scaled_df)
